# Introduction to iFEM

## Geometry of the problem : 

A beam from point A to B simply supported with length L, Youngs modulus E and Inertia I :
                                                                              
         A O___________________________________________________O   B     
          /\                      L                           /\         
         /  \                                                /  \        

nel, the number of elements for spatial discretization

In [1]:
using Muscade, StaticArrays, GLMakie, CSV, DataFrames, Interpolations
using Muscade.Toolbox

## Model creation

In [2]:
R   = 0.0;          # Radius of the bend [m]
EI₂ = 833.33e3;     # Bending stiffness [Nm²]
EI₃ = 833.33e3;     # Bending stiffness [Nm²]
EA  = 1e8;          # Axial stiffness [N]
GJ  = 705e3;        # Torsional stiffness [Nm²]
L   = 10.;           # Length of the beam [m]

nel         = 20
nnodes      = nel+1
nodeCoord   = hcat( -5. .+ ((1:nnodes).-1)/(nnodes-1)*L,
                     0  .+ zeros(Float64, nnodes, 1),
                     0  .+ zeros(Float64, nnodes, 1))
mat         = BeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.)

BeamCrossSection(1.0e8, 833330.0, 833330.0, 705000.0, 1.0, 1.0)

In [3]:
function createSimplySupportedBeam(name::Symbol; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes, eleid
end

createSimplySupportedBeam (generic function with 1 method)

In [4]:

function createDampedSimplySupportedBeam(name::Symbol, β::Float64; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    damped_mat         = DampedBeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.,β=β)
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=damped_mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes
end

createDampedSimplySupportedBeam (generic function with 1 method)

## Load scenarii
### Impulse load

In [5]:
function impulse_load(A, t, t₀; Δt = 0.05)
    abs(t-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

function impulse_load(A, t, t₀, T; Δt = 0.05)
    abs(t%T-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

impulse_load (generic function with 2 methods)

### Sinusoidal load

In [6]:
function sinus_load(A, t, T, ϕ)
    sinus_load = A * sin(2*π*t/T + ϕ)
end

function distributed_sinus_load(q, t, T_max, L, x)
    distributed_sinus_load = q*(t/T_max)*sin(2*π*x/L)
end

distributed_sinus_load (generic function with 1 method)

### Ramp load

In [7]:
function ramp_load(A, T, t)
    t₁ = T[1]
    t₂ = T[2]
    t < t₁ ? ramp_load = 0.0 :
    t < t₂ ? ramp_load = A*(t-t₁)/(t₂-t₁) :
    ramp_load = A
end

ramp_load (generic function with 1 method)

## Static analysis

In [8]:
model_stat, nodid, nnodes   = createSimplySupportedBeam(:StaticAnalysis)

Static_Loads                = [A for A in -50000.:-50000.:-150000.]
addelement!(model_stat,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t3,value= t -> Static_Loads[Int(t)] )

initialstate                = initialize!(model_stat);

loadSteps                   = [i for i in 1.:1.:length(Static_Loads)];
nLoadSteps                  = length(loadSteps)
state                       = solve(SweepX{0};initialstate,time=loadSteps,verbose=true,maxΔx=1e-9, maxiter = 50);

MethodError: MethodError: no method matching var"#DofLoad#148"(::Symbol, ::var"#31#32", ::Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, ::Type{DofLoad}, ::Vector{Node})
The function `#DofLoad#148` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  var"#DofLoad#148"(::Symbol, !Matched::Tvalue, ::Base.Pairs{Symbol, V, NTuple{N, Symbol}, NamedTuple{names, T}} where {V, N, names, T<:NTuple{N, Any}}, ::Type{DofLoad}, ::Vector{Node}) where Tvalue<:Functor
   @ Muscade C:\Users\colinsa\.julia\dev\Muscade\src\BasicElements.jl:277


In [9]:
x_ = [getdof(state[idxLoad];field=:t1,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(state[idxLoad];field=:t2,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(state[idxLoad];field=:t3,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]

fig     = Figure(size = (1000,1000))
ax      = Axis3(fig[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal, title = "Static analysis results")
clr     = [:black,:blue,:green,:red]
for idxLoad ∈ 1:nLoadSteps
    draw!(ax,state[idxLoad];EulerBeam3D=(;nseg=10, line_color = clr[idxLoad%4+1]))
end
xlims!(ax, -5,5); ylims!(ax, -5,5); zlims!(ax, -5,5);
fig

UndefVarError: UndefVarError: `nLoadSteps` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Eigenvalue analysis

In [10]:
model_eig, nodid, nnodes   = createSimplySupportedBeam(:EigAnalysis; bPlanar = true)
[addelement!(model_eig,DofLoad,[nodid[nodeidx]];field=:t2,value=t-> sinus_load(200., t, 10., 0.)) for nodeidx=1:nnodes];

initialstate    = initialize!(model_eig);
state_eig       = solve(SweepX{0};initialstate,time=[0.]);
nmod            = 15
#res             = solve(EigX{ℝ};state=state_eig[1],nmod); Assembly has some issue at this date, will have to look at it a bit more at some point


MethodError: MethodError: no method matching var"#DofLoad#148"(::Symbol, ::var"#36#38", ::Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, ::Type{DofLoad}, ::Vector{Node})
The function `#DofLoad#148` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  var"#DofLoad#148"(::Symbol, !Matched::Tvalue, ::Base.Pairs{Symbol, V, NTuple{N, Symbol}, NamedTuple{names, T}} where {V, N, names, T<:NTuple{N, Any}}, ::Type{DofLoad}, ::Vector{Node}) where Tvalue<:Functor
   @ Muscade C:\Users\colinsa\.julia\dev\Muscade\src\BasicElements.jl:277


## Dynamic analysis with various load scenarii
### Sinusoidal nodal load

In [11]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_sin)


addelement!(model_dyn,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t2,value= t -> sinus_load(1e4, t, 10., 0.) )

initialstate                = initialize!(model_dyn; time = 0.);
Tsin                        = 1.:0.05:10.
nLoadSteps                  = length(Tsin)
state                       = solve(SweepX{2};initialstate,time=Tsin,verbose=true,maxΔx=1e-8, maxiter = 80);

MethodError: MethodError: no method matching var"#DofLoad#148"(::Symbol, ::var"#39#40", ::Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, ::Type{DofLoad}, ::Vector{Node})
The function `#DofLoad#148` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  var"#DofLoad#148"(::Symbol, !Matched::Tvalue, ::Base.Pairs{Symbol, V, NTuple{N, Symbol}, NamedTuple{names, T}} where {V, N, names, T<:NTuple{N, Any}}, ::Type{DofLoad}, ::Vector{Node}) where Tvalue<:Functor
   @ Muscade C:\Users\colinsa\.julia\dev\Muscade\src\BasicElements.jl:277


In [12]:
x_sin = [[getdof(state[idxLoad];field=:t1,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
y_sin = [[getdof(state[idxLoad];field=:t2,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
z_sin = [[getdof(state[idxLoad];field=:t3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
r3_sin = [[getdof(state[idxLoad];field=:r3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in 1:10:nLoadSteps
    draw!(ax,state[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(1.0, to_draw/nLoadSteps, 0.)))
end
display(figure)
figure

UndefVarError: UndefVarError: `nLoadSteps` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Sinusoidal distributed load

In [13]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_distribsin)

q = 1e3
T_max = 10.

[addelement!(model_dyn,DofLoad,[node];field=:t3,value= t -> distributed_sinus_load(1e3, t, 10., L, nodeCoord[idx]) ) for (idx, node) in enumerate(nodid)]

initialstate                = initialize!(model_dyn; time = 0.);

loadSteps                   = [i for i in 0.05:0.05:T_max];
nLoadSteps                  = length(loadSteps)
state                       = solve(SweepX{2};initialstate,time=loadSteps,verbose=true,maxΔx=1e-8, maxiter = 80);

MethodError: MethodError: no method matching var"#DofLoad#148"(::Symbol, ::var"#46#48"{Int64}, ::Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, ::Type{DofLoad}, ::Vector{Node})
The function `#DofLoad#148` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  var"#DofLoad#148"(::Symbol, !Matched::Tvalue, ::Base.Pairs{Symbol, V, NTuple{N, Symbol}, NamedTuple{names, T}} where {V, N, names, T<:NTuple{N, Any}}, ::Type{DofLoad}, ::Vector{Node}) where Tvalue<:Functor
   @ Muscade C:\Users\colinsa\.julia\dev\Muscade\src\BasicElements.jl:277


In [14]:
figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in 1:10:nLoadSteps
    draw!(ax,state[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(1.0, to_draw/nLoadSteps, 0.)))
end
display(figure)
figure

UndefVarError: UndefVarError: `nLoadSteps` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Impulse loads
Distributed sinusoidal initial load

In [15]:
model_dyn, nodid, nnodes, eleid   = createSimplySupportedBeam(:DynAnalysis_impulse)

q = 10000
@functor with(q) load(t) = t == 0.15 ? q : 0.0
#[addelement!(model_dyn,DofLoad,[node];field=:t3,value= load) for node in nodid]
addelement!(model_dyn,DofLoad,[nodid[10]];field=:t3,value= load)
#[addelement!(model_dyn,DofLoad,[node];field=:t3,value= t -> t == 0.1 ? q*sin(2*π*nodeCoord[idx]/L) : 0.0 ) for (idx,node) in enumerate(nodid) ]

initialstate                = initialize!(model_dyn; time = 0.);
T = 0.1:0.001:3.
loadSteps                   = [i for i in T];
nLoadSteps                  = length(loadSteps)
statew                       = solve(SweepX{2};initialstate,time=loadSteps,verbose=true,maxΔx=1e-6, maxiter = 80);




Muscade: SweepX{2} solver

    step   1 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   2 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   3 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   4 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   5 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   6 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   7 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   8 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step   9 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  10 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  11 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  12 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  13 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  14 converged in   1 iterations. |Δx|=0.0e+00 |Lλ|=0.0e+00
    step  15 conve

In [16]:
req = @request gp(x∂X₀)
loads = getresult(statew[1500],req,[eleid[1]])
print(loads)

ArgumentError: ArgumentError: tuple must be non-empty

In [17]:

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in 1:10:nLoadSteps
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(to_draw/nLoadSteps, 0., 0.)))
end
display(figure)
figure

In [18]:

fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[21]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[20]]) for idxLoad ∈ 1:nLoadSteps]
z_m = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
ax = Axis(fig_zt[1, 1], xlabel="Time, t [s]", ylabel="Displacement in the y-direction [m]", title = "Direct solution to impulse load at t = 0.15s, load distributed on all nodes, No Damping")
lines!(ax, T, vcat(z_...); label="Node 20")
lines!(ax, T, vcat(z_m...); label="Node 10")
axislegend()


save("no_damping.png",fig_zt)


Local initial load

In [19]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_impulselocal)

A = 1000

addelement!(model_dyn,DofLoad,[nodid[floor(Int, nnodes/2)]];field=:t2,value= t -> t == 0.1 ? A : 0.0)

initialstate                = initialize!(model_dyn; time = 0.);

loadSteps                   = [i for i in 0.1:0.005:10.];
nLoadSteps                  = length(loadSteps)
statew                       = solve(SweepX{2};initialstate,time=loadSteps,verbose=true,maxΔx=1e-6, maxiter = 80);

MethodError: MethodError: no method matching var"#DofLoad#148"(::Symbol, ::var"#57#58", ::Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, ::Type{DofLoad}, ::Vector{Node})
The function `#DofLoad#148` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  var"#DofLoad#148"(::Symbol, !Matched::Tvalue, ::Base.Pairs{Symbol, V, NTuple{N, Symbol}, NamedTuple{names, T}} where {V, N, names, T<:NTuple{N, Any}}, ::Type{DofLoad}, ::Vector{Node}) where Tvalue<:Functor
   @ Muscade C:\Users\colinsa\.julia\dev\Muscade\src\BasicElements.jl:277


In [20]:
fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
Axis(fig_zt[1, 1])
scatter!(vcat(y_...))

fig_zt

In [21]:
figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in [10,11, 12, 13, 14]
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(to_draw/1500, 0., 0.)))
end
display(figure)
figure

## Inverse Crime

Using the data from the simulated nodal sinusoidal load

In [22]:
inv_model, nodid, nnodes = createSimplySupportedBeam(:InverseModel_bis)

T = Tsin
x_int = [linear_interpolation(T, vcat(x_sin[node]...)) for node in 1:nnodes]
y_int = [linear_interpolation(T, vcat(y_sin[node]...)) for node in 1:nnodes]
z_int = [linear_interpolation(T, vcat(z_sin[node]...)) for node in 1:nnodes]
r3_int = [linear_interpolation(T, vcat(r3_sin[node]...)) for node in 1:nnodes]

e5             = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t1,    cost= (x,t) -> 10 * (x_int[node](t)-x)^2 ) for node in 1:nnodes];
e6             = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t2,    cost= (x,t) -> 10 * (y_int[node](t)-x)^2 ) for node in 1:nnodes];
e7             = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t3,    cost= (x,t) -> 10 * (z_int[node](t)-x)^2 ) for node in 1:nnodes];
e7             = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:r3,    cost= (x,t) -> 10 * (r3_int[node](t)-x)^2 ) for node in 1:nnodes];
e2             = [addelement!(inv_model,SingleDofCost,[nodid[node]]; class=:U,field=:t3           ,    cost=(u,t) -> node == 1 ? 10*(sinus_load(1e4, t, 10., 0.)-u)^2 : 10*u^2 )  for node in 1:nnodes-1];


[[addelement!(inv_model,Hold,[nodid[i]] ;field) for field∈[:t3, :r2, :r1]] for i in 1:nnodes]

initialstateXUA    = initialize!(inv_model;time=0.)
#stateXUA         = solve(DirectXUA{2,0,0};initialstate=initialstateXUA,time=T,
#                        maxiter=100,saveiter=true,
#                        maxΔx=1e-5,maxΔλ=Inf,maxΔu=1e-5,maxΔa=1e-5);




UndefVarError: UndefVarError: `Tsin` not defined in `Main`
Suggestion: check for spelling errors or missing imports.